<a href="https://colab.research.google.com/github/kennyxu256/investigatingCryptoPumpAndDumps/blob/main/april2021DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Senior Year/busi 496/dataAnalysis/freqPumps

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive
/content/gdrive/MyDrive/Senior Year/IRP/IRPAnalysis/freqPumps


In [ ]:
!pip install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.0 MB/s eta 0:00:00


In [ ]:
import ccxt
import pandas as pd
import os

# gets symbol name from the csv file
def get_symbol(f_path):
    df = pd.read_csv(f_path, index_col=0, parse_dates=["Timestamp"])
    filename = os.path.basename(f_path)
    symbol_name = filename.split("_")[1].replace("-", "/")
    print("Loading:", symbol_name)

    return symbol_name


# extracts the symbol pairs and stores them in a df per exchange
def extract_symbol_df_from_csvs(folder):
    for subdir, dirs, files in os.walk(folder):
        symbols = []

        exchange_n = subdir.split("/")[-1]

        if 'data' not in exchange_n:
            for file in files:
                if ".csv" in file:
                    symbols.append(get_symbol('../data/' + subdir + '/' + file))
            header = ['Symbol']
            df = pd.DataFrame(symbols, columns=header)
            filename = '{}_symbols.csv'.format(exchange_n)
            df.to_csv(filename)


# analyses all the symbol pairs in subfolders of a given folder
# returns a df indexed by exchange with the number price and volume spikes, and number of pumps
def analyse_folder(folder, vol_thresh, price_thresh, window_size=24, candle_size='1h'):
    row_list = []  # list for each row of the result df

    # -- loop through folders --
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            if ".csv" in file:
                f_path = subdir + '/' + file
                result_row = analyse_symbol(f_path, vol_thresh, price_thresh, window_size, candle_size)
                row_list.append(result_row)

    # -- create result df --
    df = pd.DataFrame(row_list)
    df.set_index('Exchange', inplace=True)
    df.sort_index(inplace=True)

    return df


# Main analysis method for pump and dump detection. Returns final dataframe with number of pump and dumps.
# -- EXAMPLE INPUT --
# f_path : '../data' - output file from previously generated.
# volume_thresh : 5 (500%) - volume threshold
# price_thresh : 1.05 (5%) -  price threshold
# window_size : 24 - size of the window for the rolling average (in hours)
# candle_size = '12h' - candlesticks
# returns final dataframe
def analyse_symbol(f_path, volume_thresh, price_thresh, window_size, candle_size):
    # -- load the data --
    exchange_name, symbol_name, df = load_csv(f_path)

    # -- find spikes --
    vol_mask, vol_df = find_vol_spikes(df, volume_thresh, window_size)
    num_v_spikes = get_num_rows(vol_df)  # the number of volume spikes found for this symbol pair

    price_mask, price_df = find_price_spikes(df, price_thresh, window_size)
    num_p_spikes = get_num_rows(price_df)

    pd_mask, pd_df = find_price_dumps(df, window_size)

    vd_mask, vd_df = find_volume_dumps(df, window_size)

    # find coinciding price and volume spikes
    vp_combined_mask = (vol_mask) & (price_mask)
    vp_combined_df = df[vp_combined_mask]
    num_vp_combined_rows = get_num_rows(vp_combined_df)

    # coinciding price and volume spikes for alleged P&D (more than 1x per given time removed)
    vp_combined_rm = rm_same_day_pumps(vp_combined_df)
    num_alleged = get_num_rows(vp_combined_rm)

    # find coinciding price and volume spikes with dumps
    final_combined_mask = (vol_mask) & (price_mask) & (pd_mask)
    final_combined = df[final_combined_mask]
    final_combined_rm = rm_same_day_pumps(final_combined)  # remove indicators which occur on the same day
    num_final_combined = get_num_rows(final_combined_rm)

    row_entry = {'Exchange': exchange_name,
                 'Symbol': symbol_name,
                 'Price Spikes': num_p_spikes,
                 'Volume Spikes': num_v_spikes,
                 'Alleged Pump and Dumps': num_alleged,
                 'Pump and Dumps': num_final_combined}

    print(row_entry)

    return row_entry


def get_num_rows(df):
    return df.shape[0]


def rm_same_day_pumps(df):
    # Removes spikes that occur on the same day
    df = df.copy()
    df['Timestamp_DAYS'] = df['Timestamp'].apply(lambda x: x.replace(hour=0, minute=0, second=0))
    df = df.drop_duplicates(subset='Timestamp_DAYS', keep='last')

    return df


# finds volume spikes with a certain threshold and window size
# returns a boolean_mask, dataframe
def find_vol_spikes(df, v_thresh, win_size):
    # -- add rolling average column to df --
    vRA = str(win_size) + 'h Volume RA'
    add_RA(df, win_size, 'Volume', vRA)

    # -- find spikes --
    vol_threshold = v_thresh * df[vRA]  # v_thresh increase in volume
    vol_spike_mask = df["Volume"] > vol_threshold  # where the volume is at least v_thresh greater than the x-hr RA
    df_vol_spike = df[vol_spike_mask]

    return vol_spike_mask, df_vol_spike


# finds price spikes with a certain threshold and window size
# returns a boolean_mask, dataframe
def find_price_spikes(df, p_thresh, win_size):
    # -- add rolling average column to df --
    pRA = str(win_size) + 'h Close Price RA'
    add_RA(df, win_size, 'Close', pRA)

    # -- find spikes --
    p_threshold = p_thresh * df[pRA]  # p_thresh increase in price
    p_spike_mask = df["High"] > p_threshold  # where the high is at least p_thresh greater than the x-hr RA
    df_price_spike = df[p_spike_mask]
    return p_spike_mask, df_price_spike


# finds price dumps with a certain threshold and window size
# returns a boolean_mask, dataframe
def find_price_dumps(df, win_size):
    pRA = str(win_size) + "h Close Price RA"
    pRA_plus = pRA + "+" + str(win_size)

    df[pRA_plus] = df[pRA].shift(-win_size)
    price_dump_mask = df[pRA_plus] <= (df[pRA] + df[pRA].std())
    # if the xhour RA from after the pump was detected is <= the xhour RA (+std dev) from before the pump was detected
    # if the price goes from the high to within a range of what it was before

    df_p_dumps = df[price_dump_mask]
    return price_dump_mask, df_p_dumps


def find_volume_dumps(df, win_size):
    vRA = str(win_size) + "h Volume RA"
    vRA_plus = vRA + "+" + str(win_size)

    df[vRA_plus] = df[vRA].shift(-win_size)
    price_dump_mask = df[vRA_plus] <= (df[vRA] + df[vRA].std())
    # if the xhour RA from after the pump was detected is <= the xhour RA (+std dev) from before the pump was detected
    # if the volume goes from the high to within a range of what it was before

    df_p_dumps = df[price_dump_mask]
    return price_dump_mask, df_p_dumps


# adds a rolling average column with specified window size to a given df and col
def add_RA(df, win_size, col, name):
    df[name] = pd.Series.rolling(df[col], window=win_size, center=False).mean()


# returns a (exchange_name ,symbol_name, dataframe) tuple
def load_csv(f_path, suppress=True):
    df = pd.read_csv(f_path, index_col=0, parse_dates=["Timestamp"])
    filename = os.path.basename(f_path)
    exchange_name = filename.split("_")[0]
    symbol_name = filename.split("_")[1].replace("-", "/")

    if not suppress:
        print("Exchange:", exchange_name, "\nSymbol:", symbol_name)

    return exchange_name, symbol_name, df

In [ ]:
def countTransactions(f_path):
  df = pd.read_csv(f_path, index_col=0, parse_dates=["Timestamp"])
  return len(df.index)

In [ ]:
def countVolume(f_path):
  df = pd.read_csv(f_path, index_col=0, parse_dates=['Timestamp'])
  return df['Volume'].sum()

In [ ]:
exchangeDict = {'bittrex': [], 'kraken': [], 'kucoin': [], 'lbank': []}

In [ ]:
infoList = []
for file in os.listdir('bittrex'):
    file = 'bittrex/' + file
    infoList.append(analyse_symbol(f_path=file,
               volume_thresh=4.2,
               price_thresh=1.05,
               window_size=12,
               candle_size='1h'))
exchangeDict['bittrex'] = infoList

{'Exchange': 'bittrex', 'Symbol': '1INCH/BTC', 'Price Spikes': 9, 'Volume Spikes': 19, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 1}
{'Exchange': 'bittrex', 'Symbol': '1INCH/ETH', 'Price Spikes': 6, 'Volume Spikes': 18, 'Alleged Pump and Dumps': 1, 'Pump and Dumps': 0}
{'Exchange': 'bittrex', 'Symbol': '1INCH/USDT', 'Price Spikes': 21, 'Volume Spikes': 17, 'Alleged Pump and Dumps': 3, 'Pump and Dumps': 3}
{'Exchange': 'bittrex', 'Symbol': '4ART/BTC', 'Price Spikes': 13, 'Volume Spikes': 36, 'Alleged Pump and Dumps': 3, 'Pump and Dumps': 3}
{'Exchange': 'bittrex', 'Symbol': '4ART/USDT', 'Price Spikes': 28, 'Volume Spikes': 41, 'Alleged Pump and Dumps': 6, 'Pump and Dumps': 6}
{'Exchange': 'bittrex', 'Symbol': 'AAVE/BTC', 'Price Spikes': 18, 'Volume Spikes': 24, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 1}
{'Exchange': 'bittrex', 'Symbol': 'AAVE/ETH', 'Price Spikes': 8, 'Volume Spikes': 39, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 2}
{'Exchange': 'bittrex', 'Symbol': 'AAVE

In [ ]:
infoList = []
for file in os.listdir('kraken'):
    file = 'kraken/' + file
    infoList.append(analyse_symbol(f_path=file,
               volume_thresh=4.2,
               price_thresh=1.05,
               window_size=12,
               candle_size='1h'))
exchangeDict['kraken'] = infoList

{'Exchange': 'kraken', 'Symbol': '1INCH/EUR', 'Price Spikes': 7, 'Volume Spikes': 33, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'kraken', 'Symbol': '1INCH/USD', 'Price Spikes': 8, 'Volume Spikes': 35, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 2}
{'Exchange': 'kraken', 'Symbol': 'AAVE/BTC', 'Price Spikes': 0, 'Volume Spikes': 31, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'kraken', 'Symbol': 'AAVE/ETH', 'Price Spikes': 0, 'Volume Spikes': 48, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'kraken', 'Symbol': 'AAVE/EUR', 'Price Spikes': 9, 'Volume Spikes': 35, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'kraken', 'Symbol': 'AAVE/GBP', 'Price Spikes': 9, 'Volume Spikes': 42, 'Alleged Pump and Dumps': 1, 'Pump and Dumps': 0}
{'Exchange': 'kraken', 'Symbol': 'AAVE/USD', 'Price Spikes': 9, 'Volume Spikes': 22, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'kraken', 'Symbol': 'ACA/EUR', 'Price Sp

In [ ]:
infoList = []
for file in os.listdir('kucoin'):
    file = 'kucoin/' + file
    infoList.append(analyse_symbol(f_path=file,
               volume_thresh=4.2,
               price_thresh=1.05,
               window_size=12,
               candle_size='1h'))
exchangeDict['kucoin'] = infoList

{'Exchange': 'kucoin', 'Symbol': '1INCH/USDT', 'Price Spikes': 53, 'Volume Spikes': 2, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 1}
{'Exchange': 'kucoin', 'Symbol': 'AAVE/BTC', 'Price Spikes': 17, 'Volume Spikes': 16, 'Alleged Pump and Dumps': 1, 'Pump and Dumps': 0}
{'Exchange': 'kucoin', 'Symbol': 'AAVE/KCS', 'Price Spikes': 86, 'Volume Spikes': 8, 'Alleged Pump and Dumps': 1, 'Pump and Dumps': 1}
{'Exchange': 'kucoin', 'Symbol': 'AAVE/USDT', 'Price Spikes': 20, 'Volume Spikes': 2, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'kucoin', 'Symbol': 'ACOIN/USDT', 'Price Spikes': 78, 'Volume Spikes': 25, 'Alleged Pump and Dumps': 7, 'Pump and Dumps': 6}
{'Exchange': 'kucoin', 'Symbol': 'ADA/BTC', 'Price Spikes': 11, 'Volume Spikes': 15, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 1}
{'Exchange': 'kucoin', 'Symbol': 'ADA/KCS', 'Price Spikes': 108, 'Volume Spikes': 13, 'Alleged Pump and Dumps': 2, 'Pump and Dumps': 2}
{'Exchange': 'kucoin', 'Symbol': 'ADA/USDC', 'P

In [ ]:
infoList = []
for file in os.listdir('lbank'):
    file = 'lbank/' + file
    infoList.append(analyse_symbol(f_path=file,
               volume_thresh=4.2,
               price_thresh=1.05,
               window_size=12,
               candle_size='1h'))
exchangeDict['lbank'] = infoList

{'Exchange': 'lbank', 'Symbol': '1INCH/USDT', 'Price Spikes': 52, 'Volume Spikes': 5, 'Alleged Pump and Dumps': 1, 'Pump and Dumps': 0}
{'Exchange': 'lbank', 'Symbol': '1INCH3L/USDT', 'Price Spikes': 214, 'Volume Spikes': 3, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'lbank', 'Symbol': '1INCH3S/USDT', 'Price Spikes': 160, 'Volume Spikes': 5, 'Alleged Pump and Dumps': 3, 'Pump and Dumps': 3}
{'Exchange': 'lbank', 'Symbol': '1INCH5L/USDT', 'Price Spikes': 247, 'Volume Spikes': 33, 'Alleged Pump and Dumps': 11, 'Pump and Dumps': 11}
{'Exchange': 'lbank', 'Symbol': '1INCH5S/USDT', 'Price Spikes': 214, 'Volume Spikes': 13, 'Alleged Pump and Dumps': 5, 'Pump and Dumps': 4}
{'Exchange': 'lbank', 'Symbol': '3ULL/USDT', 'Price Spikes': 53, 'Volume Spikes': 1, 'Alleged Pump and Dumps': 0, 'Pump and Dumps': 0}
{'Exchange': 'lbank', 'Symbol': '4JNET/USDT', 'Price Spikes': 175, 'Volume Spikes': 6, 'Alleged Pump and Dumps': 1, 'Pump and Dumps': 1}
{'Exchange': 'lbank', 'Symbol': 

In [ ]:
exchangeDict

{'bittrex': [{'Exchange': 'bittrex',
   'Symbol': '1INCH/BTC',
   'Price Spikes': 9,
   'Volume Spikes': 19,
   'Alleged Pump and Dumps': 2,
   'Pump and Dumps': 1},
  {'Exchange': 'bittrex',
   'Symbol': '1INCH/ETH',
   'Price Spikes': 6,
   'Volume Spikes': 18,
   'Alleged Pump and Dumps': 1,
   'Pump and Dumps': 0},
  {'Exchange': 'bittrex',
   'Symbol': '1INCH/USDT',
   'Price Spikes': 21,
   'Volume Spikes': 17,
   'Alleged Pump and Dumps': 3,
   'Pump and Dumps': 3},
  {'Exchange': 'bittrex',
   'Symbol': '4ART/BTC',
   'Price Spikes': 13,
   'Volume Spikes': 36,
   'Alleged Pump and Dumps': 3,
   'Pump and Dumps': 3},
  {'Exchange': 'bittrex',
   'Symbol': '4ART/USDT',
   'Price Spikes': 28,
   'Volume Spikes': 41,
   'Alleged Pump and Dumps': 6,
   'Pump and Dumps': 6},
  {'Exchange': 'bittrex',
   'Symbol': 'AAVE/BTC',
   'Price Spikes': 18,
   'Volume Spikes': 24,
   'Alleged Pump and Dumps': 2,
   'Pump and Dumps': 1},
  {'Exchange': 'bittrex',
   'Symbol': 'AAVE/ETH',
   'P

In [ ]:
allegedPD  = {'bittrex': 0, 'kraken': 0, 'kucoin': 0, 'lbank': 0}
confirmedPD = {'bittrex': 0, 'kraken': 0, 'kucoin': 0, 'lbank': 0}

In [ ]:
for exchange in allegedPD.keys():
  for dictionary in exchangeDict[exchange]:
    allegedPD[exchange] += dictionary['Alleged Pump and Dumps']
    confirmedPD[exchange] += dictionary['Pump and Dumps']

print(allegedPD)
print(confirmedPD)

{'bittrex': 2020, 'kraken': 1566, 'kucoin': 156, 'lbank': 1712}
{'bittrex': 1624, 'kraken': 1293, 'kucoin': 124, 'lbank': 1522}


In [ ]:
totalCount = {'bittrex': 0, 'kraken': 0, 'kucoin': 0, 'lbank': 0}

In [ ]:
for exchange in totalCount.keys():
  for file in os.listdir(exchange):
    file = exchange + '/' + file
    totalCount[exchange] += countTransactions(f_path = file)

In [ ]:
percentagePD = {'bittrex': 0, 'kraken': 0, 'kucoin': 0, 'lbank': 0}
for exchange in percentagePD.keys():
  percentagePD[exchange] = (confirmedPD[exchange] / totalCount[exchange])

print(percentagePD)

{'bittrex': 0.006519548609577795, 'kraken': 0.00420408508313879, 'kucoin': 0.0037302208050057155, 'lbank': 0.003361656355672961}


In [ ]:
allPDCount = []
for exchange in exchangeDict:
  for dictionary in exchangeDict[exchange]:
    allPDCount.append((dictionary['Pump and Dumps'], dictionary['Exchange']))

In [ ]:
allPDCount.sort(reverse=True)
print(allPDCount[0:49])

[(15, 'lbank'), (13, 'lbank'), (13, 'lbank'), (13, 'bittrex'), (12, 'lbank'), (12, 'lbank'), (12, 'lbank'), (12, 'lbank'), (12, 'bittrex'), (11, 'lbank'), (11, 'lbank'), (11, 'lbank'), (11, 'lbank'), (11, 'lbank'), (11, 'lbank'), (11, 'kraken'), (11, 'bittrex'), (11, 'bittrex'), (11, 'bittrex'), (11, 'bittrex'), (11, 'bittrex'), (10, 'lbank'), (10, 'lbank'), (10, 'lbank'), (10, 'lbank'), (10, 'lbank'), (10, 'lbank'), (10, 'lbank'), (10, 'bittrex'), (10, 'bittrex'), (10, 'bittrex'), (10, 'bittrex'), (10, 'bittrex'), (10, 'bittrex'), (10, 'bittrex'), (9, 'lbank'), (9, 'lbank'), (9, 'lbank'), (9, 'lbank'), (9, 'lbank'), (9, 'lbank'), (9, 'lbank'), (9, 'kraken'), (9, 'kraken'), (9, 'kraken'), (9, 'kraken'), (9, 'kraken'), (9, 'kraken'), (9, 'bittrex')]


In [ ]:
volumeSum = {'bittrex': 0, 'kraken': 0, 'kucoin': 0, 'lbank': 0}

In [ ]:
for exchange in totalCount.keys():
  for file in os.listdir(exchange):
    file = exchange + '/' + file
    volumeSum[exchange] += countVolume(f_path = file)

In [ ]:
volumeSum

{'bittrex': 29389496608.89803,
 'kraken': 18485052549674.195,
 'kucoin': 137619532685.21765,
 'lbank': 6.9553797219162e+17}

In [ ]:
volumePercentPD = {'bittrex': 0, 'kraken': 0, 'kucoin': 0, 'lbank': 0}
for exchange in volumePercentPD.keys():
  volumePercentPD[exchange] = (confirmedPD[exchange] / volumeSum[exchange])

print(volumePercentPD)

{'bittrex': 5.525783655335948e-08, 'kraken': 6.994840812734338e-11, 'kucoin': 9.010348864040243e-10, 'lbank': 2.188234231416902e-15}
